# Problem set 2
Before we start working on some exercises we will briefly introduce two concepts in Python. First, importing and exporting data. Second, using functions. If you are already familiar
with these features, you can skip the next two sections and jump directly to the exercises.

First, import all necessary packages. We have made a .py file that we will use as a "toolbox". We will fill this toolbox with functions, that we will use as we progress through the course. Exactly how you structure this toolbox is up to you (if you i.e. want to turn it into a class).

In [1]:
import numpy as np
from numpy import linalg as la
import pandas as pd
from io import StringIO
from tabulate import tabulate
from matplotlib import pyplot as plt

# Import this weeks LinearModels .py file
import LinearModelsPS2_post as lm
%load_ext autoreload
%autoreload 2

## Importing and exporting data in Python
The easiest way to import data into an numpy array is using a .txt file. Normally we specify a path to the text file, but we will create a fake one to illustrate.

In [2]:
# Create a fake file for easy use.
fake_file = StringIO("0 1\n 2 3")
print(f"Fake file looks like this: \n {fake_file.getvalue()}")
print()

# Load the fake txt file into a numpy array.
data = np.loadtxt(fake_file)
print(f'Loaded into a numpy array, we get the following {type(data)}: \n {data}')

Fake file looks like this: 
 0 1
 2 3

Loaded into a numpy array, we get the following <class 'numpy.ndarray'>: 
 [[0. 1.]
 [2. 3.]]


Sadly, there is no direct way to load an excel sheet into numpy. The easiest solution is to use pandas as an intermediate.

In [3]:
# We save the fake file we created earlier as an excel file, 
# so that we can illustrate how to import using excel.
to_export = pd.DataFrame(data)
to_export.to_excel('test_file.xlsx', header=None, index=None)

# Its important to note that Pandas will treat the first row as a header. If there is no header,
# this needs to be specified. There are also alot of extra options to load specific sheets, or
# only parts of the sheets and tons of extra options.
df_import = pd.read_excel('test_file.xlsx', header=None)
np_array = df_import.to_numpy()
print(np_array)

[[0 1]
 [2 3]]


### Exporting Data
To save a numpy array as a .txt file is easy:

In [4]:
np.savetxt('real_file.txt', np_array)
print()

*If one has large numpy arrays and wants to store them efficiently, they can be saved as a binary .npy files. Such files are not compatible with other programs.*

## Exercises with FE - Within-Groups Estimation

The exercise takes up the union membership example from before. The data set WAGEPAN.TXT contains information about 545 men who worked every year from 1980 to 1987 in the US. The variables of interest are


| Variable | Content |
|-|-|
| nr | Variable that identifies the individual  |
| year | Year of observation |
| Black | Black |
| Hisp | Hispanic |
| Educ | Years of schooling |
| Exper | Years since left school |
| Expersq | Exper2 |
| Married | Marital status |
| Union | Union membership |
| Lwage | Natural logarithm of hourly wages |

Consider the following wage equation:

$$
\begin{align}
\log\left(wage_{it}\right) & =\beta_{0}+\beta_{1}\textit{exper}_{it}+\beta_{2}\textit{exper}_{it}^{2}+\beta_{3}\textit{union}_{it}+\beta_{4}\textit{married}_{i} +\beta_{5}\textit{educ}_{i}+\beta_{6}\textit{hisp}_{i}+\beta_{7}\textit{black}_{i}+c_{i}+u_{it} \tag{1}
\end{align}
$$

Note that *educ*, *hisp*, and *black* are time-invariant variables.

## FE Questions
### FE (a):
- **Estimate (1) by pooled OLS,** thus considering for the moment the unobserved components of (1) as one (composite) error term $v_{it}=c_{i}+u_{it}$. 
- What assumptions are made about $E\left[c_{i}\mathbf{x}_{it}\right]$ and $E\left[u_{it}\mathbf{x}_{it}\right]$ when justifying this estimation approach?

In [5]:
# First, import the data into numpy. 
data = np.loadtxt('wagepan.txt', delimiter=",")
id_array = np.array(data[:, 0])

# Count how many persons we have. This returns a tuple with the unique IDs,
# and the number of times each person is observed.
unique_id = np.unique(id_array, return_counts=True)
N = unique_id[0].size
T = int(unique_id[1].mean())
year = np.array(data[:, 1], dtype=int)

The table above does not correspond 1:1 with the text file. The data has 10 columns. Named from 0 to 9. Here is a variable describtion:
- Column 0: ID
- Column 1: Year
- Column 2: Black
- Column 3: Experience
- Column 4: Hispanic
- Column 5: Married
- Column 6: Education
- Column 7: Union
- Column 8: ln wage
- Column 9: Experience sqr

In [6]:
# Load the rest of the data into arrays.
y = np.array(data[:, 8]).reshape(-1, 1)
x = np.array(
    [np.ones((y.shape[0])), 
    data[:, 2],
    data[:, 4],
    data[:, 6],
    data[:, 3],
    data[:, 9],
    data[:, 5],
    data[:, 7]]
).T

# Lets also make some variable names
label_y = 'Log wage'
label_x = [
    'Constant', 
    'Black', 
    'Hispanic', 
    'Education', 
    'Experience', 
    'Experience sqr', 
    'Married', 
    'Union'
]

In [7]:
ols_result = lm.estimate(y, x,N=N,T=T)
lm.print_table(
    (label_y, label_x), ols_result, title="Pooled OLS", floatfmt='.4f'
)

Pooled OLS
Dependent variable: Log wage

                   Beta      Se    t-values
--------------  -------  ------  ----------
Constant        -0.0347  0.0646     -0.5375
Black           -0.1438  0.0236     -6.1055
Hispanic         0.0157  0.0208      0.7543
Education        0.0994  0.0047     21.2476
Experience       0.0892  0.0101      8.8200
Experience sqr  -0.0028  0.0007     -4.0272
Married          0.1077  0.0157      6.8592
Union            0.1801  0.0171     10.5179
R² = 0.187
σ² = 0.231


You should get a table that looks like this:

Pooled OLS <br>
Dependent variable: Log wage <br>

|                |    Beta |     Se |   t-values |
|----------------|---------|--------|------------|
| Constant       | -0.0347 | 0.0646 |    -0.5375 |
| Black          | -0.1438 | 0.0236 |    -6.1055 |
| Hispanic       |  0.0157 | 0.0208 |     0.7543 |
| Education      |  0.0994 | 0.0047 |    21.2476 |
| Experience     |  0.0892 | 0.0101 |     8.8200 |
| Experience sqr | -0.0028 | 0.0007 |    -4.0272 |
| Married        |  0.1077 | 0.0157 |     6.8592 |
| Union          |  0.1801 | 0.0171 |    10.5179 |
R² = 0.187 <br>
σ² = 0.231

### FE (b):
- Within transform the data. What happens to *educ, hisp, and black* and $x_{it1}\equiv1$ when the data are within transformed? 
- What is the rank of the within transformed $\mathbf{X}$ matrix? Why?

In [8]:
def demeaning_matrix(T):
    Q_T = np.eye(T) - np.tile(1/T, (T, T))
    return Q_T


In [9]:
Q_T = demeaning_matrix(T)
y_demean = lm.perm(Q_T, y)
x_demean = lm.perm(Q_T, x)

# Check rank of demeaned matrix, and return its eigenvalues.
def check_rank(x):
    print(f'Rank of demeaned x: {la.matrix_rank(x)}')
    lambdas, V = la.eig(x.T@x)
    np.set_printoptions(suppress=True)  # This is just to print nicely.
    print(f'Eigenvalues of within-transformed x: {lambdas.round(decimals=0)}')
check_rank(x_demean)

# We need to drop the columns that are constant over time.
# Try to first estimate the OLS without removing the 0-columns.
x_demean = x_demean[:, 4:]
label_x_fe = label_x[4:]

Rank of demeaned x: 4
Eigenvalues of within-transformed x: [4248875.    1871.     365.     329.       0.       0.       0.       0.]


An alternative to the perm function is to use panda dataframes to group the data. This has been done for the *entire* dataset below. If you want to use this method you'll need to make sure to pull out the correct variables when computing first difference and fixed effects estimators.

In [10]:
# load the data using pandas
pddat = pd.read_csv('wagepan.txt', delimiter=",", header=None)
pddat.columns = ["ID", "year", "black", "exp", "hisp", "mar", "educ", "union", "logwage", "expsq"]

# first difference the data
pddat = pddat.sort_values(["ID", "year"]) # important to ensure years are sorted correctly
pddat_diff = pddat.groupby("ID").diff().dropna() # take differences and drop NaNs due to differencing
datdiff = pddat_diff.to_numpy() # turn into numpy array

# demean data
pddat_demean=pddat-pddat.groupby("ID").transform('mean')
datdemean = pddat_demean.to_numpy() # turn into numpy array

In [11]:
# Can look at the dataframe to see where the relevant variables are, note dimensions
pddat_diff

,year,black,exp,hisp,mar,educ,union,logwage,expsq
1,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.655520,3.0
2,1.0,0.0,1.0,0.0,0.0,0.0,-1.0,-0.508598,5.0
3,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.088752,7.0
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.134912,9.0
5,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.131766,11.0
...,...,...,...,...,...,...,...,...,...
4355,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.759397,15.0
4356,1.0,0.0,1.0,0.0,0.0,0.0,1.0,-0.379336,17.0
4357,1.0,0.0,1.0,0.0,0.0,0.0,-1.0,0.553419,19.0
4358,1.0,0.0,1.0,0.0,0.0,0.0,1.0,-0.020068,21.0


In [12]:
ydiff = np.array(datdiff[:, 7]).reshape(-1, 1)
xdiff = np.array([
    datdiff[:, 2],
    datdiff[:, 8],
    datdiff[:, 4],
    datdiff[:, 6]]
).T
la.matrix_rank(xdiff)

4

In [13]:
# Can look at the dataframe to see where the relevant variables are, note dimensions
pddat_demean

,ID,black,educ,exp,expsq,hisp,logwage,mar,union,year
0,NaN,0.0,0.0,-3.5,-24.5,0.0,-0.058112,0.000,-0.125,-3.5
1,NaN,0.0,0.0,-2.5,-21.5,0.0,0.597408,0.000,0.875,-2.5
2,NaN,0.0,0.0,-1.5,-16.5,0.0,0.088810,0.000,-0.125,-1.5
3,NaN,0.0,0.0,-0.5,-9.5,0.0,0.177561,0.000,-0.125,-0.5
4,NaN,0.0,0.0,0.5,-0.5,0.0,0.312473,0.000,-0.125,0.5
...,...,...,...,...,...,...,...,...,...,...
4355,NaN,0.0,0.0,-0.5,-13.5,0.0,0.209697,0.375,-0.375,-0.5
4356,NaN,0.0,0.0,0.5,3.5,0.0,-0.169639,0.375,0.625,0.5
4357,NaN,0.0,0.0,1.5,22.5,0.0,0.383780,0.375,-0.375,1.5
4358,NaN,0.0,0.0,2.5,43.5,0.0,0.363713,0.375,0.625,2.5


In [14]:
y_dot = np.array(datdemean[:, 6]).reshape(-1, 1)
x_dot = np.array([
    datdemean[:, 3],
    datdemean[:, 4],
    datdemean[:, 7],
    datdemean[:, 8]]
).T

### FE (c):
- Estimate (1) on within transformed data (make sure that the employed $\mathbf{\ddot{X}}$ has full rank - drop columns if necessary). 
- How big is the union premium according to the estimate from the FE model? Compare this with the estimate that you calculated from the pooled OLS regression. What does this suggest about $E\left[union_{it}c_{i}\right]$?

In [15]:
# Estimate FE OLS using the demeaned variables.
fe_result = lm.estimate(
    y_dot, x_dot, transform='fe', N=N, T=T # could also use y_demean, x_demean from perm function
)
lm.print_table(
    (label_y, label_x_fe), 
    fe_result, title='FE regression', floatfmt='.4f'
)

FE regression
Dependent variable: Log wage

                   Beta      Se    t-values
--------------  -------  ------  ----------
Experience       0.1168  0.0084     13.8778
Experience sqr  -0.0043  0.0006     -7.1057
Married          0.0453  0.0183      2.4743
Union            0.0821  0.0193      4.2553
R² = 0.178
σ² = 0.123


You should get a table that looks like this:

FE regression<br>
Dependent variable: Log wage

|                |    Beta |     Se |   t-values |
|----------------|---------|--------|------------|
| Experience     |  0.1168 | 0.0084 |    13.8778 |
| Experience sqr | -0.0043 | 0.0006 |    -7.1057 |
| Married        |  0.0453 | 0.0183 |     2.4743 |
| Union          |  0.0821 | 0.0193 |     4.2553 |
R² = 0.178 <br>
σ² = 0.123

## FD Questions
### FD (a):
- Construct $\mathbf{D}$ and use the procedure `perm` $(\mathbf{D},\mathbf{x})$ to compute first differences of the elements of $\mathbf{y}$ and $\mathbf{x}$. 
- What happens to *educ, hisp* and *black* and $x_{it1}\equiv1$ when the data are transformed into first differences? What is the rank of the first differenced $\mathbf{x}$-matrix? Why?

In [16]:
def fd_matrix(T):
    D_T = np.eye(T) - np.eye(T, k=-1)
    D_T = D_T[1:]
    return D_T

In [17]:
# Transform the data.
D_T = fd_matrix(T)
y_diff = lm.perm(D_T, y)
x_diff = lm.perm(D_T, x)

# Again, check rank condition.
check_rank(x_diff)

# Remember to remove linear dependent columns
x_diff = x_diff[:, 4:]

Rank of demeaned x: 4
Eigenvalues of within-transformed x: [753711.    356.    545.    508.      0.      0.      0.      0.]


### FD (b):
- Estimate (1) in first differences. How big is the union premium according to the estimate from this model? 
- Compare the FD estimate with the estimate that you calculated from the FE regression. Is there a difference? If yes, what (if anything) can we conclude based on this finding?

In [18]:
fd_result = lm.estimate(
    ydiff, xdiff, N=N, T=T-1 # could also use y_diff, x_diff from perm function
)
lm.print_table(
    (label_y, label_x_fe), 
    fd_result, title='FD regression', floatfmt='.4f'
)

FD regression
Dependent variable: Log wage

                   Beta      Se    t-values
--------------  -------  ------  ----------
Experience       0.1158  0.0196      5.9096
Experience sqr  -0.0039  0.0014     -2.8005
Married          0.0381  0.0229      1.6633
Union            0.0428  0.0197      2.1767
R² = 0.004
σ² = 0.196


You should get a table that looks like this:

FD regression <br>
Dependent variable: Log wage

|                |    Beta |     Se |   t-values |
|----------------|---------|--------|------------|
| Experience     |  0.1158 | 0.0196 |     5.9096 |
| Experience sqr | -0.0039 | 0.0014 |    -2.8005 |
| Married        |  0.0381 | 0.0229 |     1.6633 |
| Union          |  0.0428 | 0.0197 |     2.1767 |
R² = 0.004 <br>
σ² = 0.196

## Exercise comparing FE and FD
### Question FE v. FD (a):
**Test for serial correlation in the errors using an auxilliary AR(1) model**, to test assumption FD.3, where the errors $e_{it} = \Delta u_{it}$ should be serially uncorrelated.

We can easily test this assumption given the OLS residuals from the FD version of equation (1). Run the regression (note that you will loose data for
the *two* first periods)
\begin{equation}
\hat{e}_{it}=\rho\hat{e}_{it-1}+error_{it},\quad t=\color{red}{3},\dotsc,T,\quad i=1,\dotsc,N\tag{2}
\end{equation}

Do you find any evidence for serial correlation? Does FD.3 seem appropriate? And why don't we include an intercept in this auxilliary equation?

*Note:* Under FE.3, the idiosyncratic errors $u_{it}$
are uncorrelated. However, FE.3 implies that the $e_{it}$'s are autocorrelated. In fact, of the $u_{it}$'s are serially uncorrelated to beging with, corr$\left(e_{it},e_{it-1}\right)=-0.5$. (Check!) This test is of course only valid if the explanatory variables are strictly exogenous!

*Hint:* You can use the `perm` function to lag
the error term variable. Consider the following; 

$$
{\begin{bmatrix}
1 & 0 & 0 & \cdots & 0 & 0\\
0 & 1 & 0 & \cdots & 0 & 0\\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots\\
0 & 0 & 0 & \cdots & 1 & 0
\end{bmatrix}}_{T-1\times T}\times{\begin{bmatrix}y_{1}\\
y_{2}\\
\vdots\\
y_{T}
\end{bmatrix}}_{T \times 1}={\begin{bmatrix}y_{1}\\
y_{2}\\
\vdots\\
y_{T - 1}
\end{bmatrix}}_{T - 1\times 1}
$$

In [19]:
def serial_corr(y, x, T, year):
    b_hat = lm.est_ols(y, x)
    e = y - x@b_hat
    
    # Create a lag to estimate the error on.
    L_T = np.eye(T, k=-1)
    L_T = L_T[1:]

    e_l = lm.perm(L_T, e)

    # We then need to remove the first obs for every person again.
    reduced_year = year[year != 1980]
    e = e[reduced_year != 1981]

    return lm.estimate(e, e_l,N=N,T=T-1)

In [20]:
corr_result = serial_corr(y_diff, x_diff, T-1, year)

label_ye = 'OLS residual, e\u1d62\u209c'
label_e = ['e\u1d62\u209c\u208B\u2081']
title = 'Serial Correlation'
lm.print_table(
    (label_ye, label_e), corr_result, 
    title='Serial Correlation', floatfmt='.4f'
)

Serial Correlation
Dependent variable: OLS residual, eᵢₜ

          Beta      Se    t-values
-----  -------  ------  ----------
eᵢₜ₋₁  -0.3961  0.0147    -27.0185
R² = 0.182
σ² = 0.143


You should get a table that looks like this:

Serial Correlation <br>
Dependent variable: OLS residual, eᵢₜ

|       |    Beta |     Se |   t-values |
|-------|---------|--------|------------|
| eᵢₜ₋₁ | -0.3961 | 0.0147 |   -27.0185 |
R² = 0.182 <br>
σ² = 0.143

### Question FE v FD (b):

Test for strict exogeneity: Add a lead of the union variable, $union_{i,t+1}$ to equation (1) (note that you will lose data from period $T$ , 1987) and estimate the model with *fixed effects* (i.e., you have to demean $union_{i,t+1}$ along with all the other variables and throw out time constant variables). Is $union_{i,t+1}$ significant? What does this imply for the strict exogeneity assumption?

*Hint:* To lead a variable, think along the same lines as in Question FE v FD (a)

In [21]:
def exogeneity_test(x, y, T):
    # Create lead
    F_T = np.eye(T, k=1)
    F_T = F_T[:-1]

    # Lead union
    union_lead = lm.perm(F_T, x[:, 7].reshape(-1, 1))

    # Collect variables to test for exogeneity
    x_exo = x[year != 1987]
    x_exo = np.hstack((x_exo, union_lead))
    y_exo = y[year != 1987]

    # Within transform the data
    Q_T = demeaning_matrix(T - 1)

    yw_exo = lm.perm(Q_T, y_exo)
    xw_exo = lm.perm(Q_T, x_exo)
    xw_exo = xw_exo[:, 4:]

    label_exo = label_x_fe + ['Union lead']
    n = y.size/T
    # Estimate model
    exo_test = lm.estimate(
        yw_exo, xw_exo, N=N, T=T - 1, transform='fe'
    )
    # Adjust sigma2 since

    lm.print_table(
        (label_y, label_exo), 
        exo_test, title='Exogeneity test', floatfmt='.4f'
    )
exogeneity_test(x, y, T)

Exogeneity test
Dependent variable: Log wage

                   Beta      Se    t-values
--------------  -------  ------  ----------
Experience       0.1213  0.0100     12.1001
Experience sqr  -0.0050  0.0008     -6.3579
Married          0.0436  0.0209      2.0898
Union            0.0757  0.0218      3.4784
Union lead       0.0515  0.0223      2.3063
R² = 0.146
σ² = 0.128


The table should look something like this:
Exogeneity test <br>
Dependent variable: Log wage

|                |    Beta |     Se |   t-values |
|----------------|---------|--------|------------|
| Experience     |  0.1213 | 0.0100 |    12.1001 |
| Experience sqr | -0.0050 | 0.0008 |    -6.3579 |
| Married        |  0.0436 | 0.0209 |     2.0898 |
| Union          |  0.0757 | 0.0218 |     3.4784 |
| Union lead     |  0.0515 | 0.0223 |     2.3063 |
R² = 0.146<br>
σ² = 0.128

### Question FE v FD (b):
Add interactions on the form $d_{81}\cdot educ, d_{82}\cdot educ, ..., d_{87}\cdot educ$ and estimate the model with fixed effect. Has the return to education increased over time?

*Hint:* Remember that $educ_{i}$ doesn't vary over
time! Therefore we didn't use $educ$ in levels in the FE estimation.
However, if we suppose that the structural equation (4) contains a term $\sum_{s=2}^{T}\delta_{s}d_{s}educ_{i}$, it will be perfectly fine to within-transform these interactions since they vary over time (although in a highly structured manner - they equal
zero in all time periods but one, and then $educ$). Note that one
period is dropped for the within-transformation to work whereas the
levels term, $\beta_{5}educ_{i}$, is dropped to avoid producing a
constant row.

*Programming hint:* You want to append the dataset with a dummy matrix, that would look something like this:

$$
\begin{bmatrix}
0 & 0 & 0 & 0 & 0 & 0 & 0 \\
14 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 14 & 0 & 0 & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & 0 & 0 & 0 & 0 & 14 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 \\
9 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 9 & 0 & 0 & 0 & 0 & 0 \\
\vdots & \vdots & \ddots & \vdots & \vdots & \vdots & \vdots \\
0 & 0 & 0 & 0 & 0 & 0 & 9 \\
\end{bmatrix}
$$

This example shows our two first persons, that have 14 and 9 years of education respectively. This matrix can be created as a product of two matrices, what would they look like (a dummy matrix with onlyn ones can be creates using `np.eye` and `np.tile`)? Why is the first row for each person only zeros?

In [22]:

# This dummy block has a 0 row, as we need to exclude one
# year in order to not end up in the dummy trap.
dummy_block = np.eye(T, k=-1)[:, :-1]

# Expand thid dummy block to all persons
dummy_matrix = np.tile(dummy_block, (N, 1))

# We now create a n*t-1 matrix, with the person's education 
expanded_educ = np.transpose([x[:, 3]] * (T-1))

# We can now multiplu the year dummy with a person's education
educ_dummies = dummy_matrix*expanded_educ
educ_demean = lm.perm(Q_T, educ_dummies)
x_demean_dummies = np.hstack([x_demean, educ_demean])

label_x_interactions = label_x_fe + [
    'E81', 'E82', 'E83', 'E84', 'E85', 'E86', 'E87'
]

In [23]:
int_result = lm.estimate(
    y_demean, x_demean_dummies, transform='fe', N=N, T=T
)
lm.print_table(
    (label_y, label_x_interactions), int_result, 
    title='FE with year interactions', floatfmt='.4f'
)

FE with year interactions
Dependent variable: Log wage

                   Beta      Se    t-values
--------------  -------  ------  ----------
Experience       0.1705  0.0273      6.2462
Experience sqr  -0.0060  0.0009     -6.9581
Married          0.0475  0.0183      2.5925
Union            0.0794  0.0193      4.1138
E81             -0.0010  0.0026     -0.4009
E82             -0.0062  0.0041     -1.5224
E83             -0.0114  0.0057     -2.0006
E84             -0.0136  0.0072     -1.8787
E85             -0.0162  0.0087     -1.8578
E86             -0.0170  0.0101     -1.6804
E87             -0.0167  0.0115     -1.4619
R² = 0.181
σ² = 0.123


You should get a table that looks like this:

FE with year interactions <br>
Dependent variable: Log wage

|                |    Beta |     Se |   t-values |
|----------------|---------|--------|------------|
| Experience     |  0.1705 | 0.0273 |     6.2462 |
| Experience sqr | -0.0060 | 0.0009 |    -6.9581 |
| Married        |  0.0475 | 0.0183 |     2.5925 |
| Union          |  0.0794 | 0.0193 |     4.1138 |
| E81            | -0.0010 | 0.0026 |    -0.4009 |
| E82            | -0.0062 | 0.0041 |    -1.5224 |
| E83            | -0.0114 | 0.0057 |    -2.0006 |
| E84            | -0.0136 | 0.0072 |    -1.8787 |
| E85            | -0.0162 | 0.0087 |    -1.8578 |
| E86            | -0.0170 | 0.0101 |    -1.6804 |
| E87            | -0.0167 | 0.0115 |    -1.4619 |
R² = 0.181 <br>
σ² = 0.123